In [26]:
from gensim.models import Word2Vec
import numpy as np
import random
import re
import os
import torch
import pandas as pd
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import random

model = Word2Vec.load('wv_sentiment_aclimdb_len50')
POS_TRAIN_PATH = 'data/aclImdb/train/pos/'
NEG_TRAIN_PATH = 'data/aclImdb/train/neg/'
pos_files = os.listdir(POS_TRAIN_PATH)
neg_files = os.listdir(NEG_TRAIN_PATH)
print('Docs prep....')
pos_lines = []
neg_lines = []
for file in pos_files:
    with open(POS_TRAIN_PATH + file) as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines if line.strip()]
        pos_lines.extend(lines)
for file in neg_files:
    with open(NEG_TRAIN_PATH + file) as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines if line.strip()]
        neg_lines.extend(lines)

docs = []
docs.extend(pos_lines)
docs.extend(neg_lines)

pos_labels = np.zeros(len(pos_lines))
neg_labels = np.ones(len(neg_lines))

print(len(docs))
docs = [re.sub('([0-9\-|\?!]+|(<br\s?\/?>))',' ',sent) for sent in docs]
print(len(docs))
df2 = pd.DataFrame(docs)
labels2 = []
labels2 = np.append(pos_labels, neg_labels)
df2['labels'] = labels2
df2 = df2.sample(frac=1)
def word_to_tensor(word):
    return model.wv[word]

def word_to_tensor(word):
    return model.wv[word]

def sent_to_tensor(sent):
    tokens = word_tokenize(sent)
    tokens = list(filter(lambda x:x in model.wv.vocab,tokens))
    arr = []
    for token in tokens:
        arr.append(word_to_tensor(token))
    return np.array(arr)

def  getRand():
    index = random.randint(0,len(X_train)-1)
    sent = df2.iloc[index][0]
    label = df2.iloc[index][1]
    return sent, label

# df2.to_csv('data/aclImdb/processed_input_labelled3.csv')

# pos_labels = np.zeros(len(pos_lines))
# neg_labels = np.ones(len(neg_lines))

X = df2.drop('labels', axis=1)
X = np.array(X)
print(X.shape)
train_size = int(len(docs)*.80)
X_train = X[:train_size]
X_test = X[train_size:]
y_train = df2['labels'][:train_size]
y_test = df2['labels'][train_size:]

# torch.save(rnn.state_dict(), 'rnn.pth') ######### uncomment later


Docs prep....
25000
25000
(25000, 1)


In [28]:
def  getVal(index):
#     index = random.randint(0,len(X_train)-1)
    sent = df2.iloc[index][0]
    label = df2.iloc[index][1]
    return sent, label

In [24]:
print(train_size)

5000


In [6]:
X_test = X[train_size:]

In [29]:
sent, label = getVal(0)
print(sent, label)
input = sent_to_tensor(sent)
print(torch.Tensor(input))
label = torch.from_numpy(np.array([label]))
print(label)


This film was a disaster from start to finish. Interspersed with performances from "the next generation of beautiful losers" are interviews with Bono and The Edge as well as the performers themselves. This leaves little time for the clips of Leonard Cohen himself, who towers over everyone else in the film with his commanding yet gentle presence, wisdom and humor. The rest are too busy trying to canonize him as St. Leonard or as some Old Testament prophet. Many of the performances are forgettable over interpretations (especially Rufus & Martha Wainright's) or bland under achievements. Only Beth Orton and Anthony got within striking distance of Leonard's own versions by using a little restraint. Annoying little pseudo avant garde gestures are sprinkled throughout the film  like out of focus superimpositions of red spheres over many of the concert and interview shots, shaky blurred camera work, use of digital delay on some of Leonard Cohen's comments (making it harder to hear what's being

In [30]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim , hidden_dim,  target_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
#         self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        return (Variable(torch.zeros(1, 1, self.hidden_dim)), 
                Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, sentence):
#         print('IN FORWARD PASS')
#         embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(sentence.view(len(sentence), 1, -1), self.hidden)
#         print('EMBED size:', embeds.size())
#         print('LSTM out: ', lstm_out.size())
#         print('LAST :', lstm_out[-1].size())
#         print('RESIZED FOR LINEAR')
#         print(lstm_out.view(len(sentence), -1).size())
        tag_space = self.hidden2tag(lstm_out[-1])
#         print('TAG SPACE', tag_space.size())
        tag_scores = F.log_softmax(tag_space)
#         print('TAG SCORE size: ',tag_scores.size())
        return tag_scores

In [31]:
lstmmodel = LSTMTagger(50, 128, 2)

In [32]:
lstmmodel(Variable(torch.Tensor(input)))

Variable containing:
-0.6690 -0.7179
[torch.FloatTensor of size 1x2]

In [33]:
import torch.optim as optim

In [104]:
%%time
EMBEDDING_DIM = 50
HIDDEN_DIM = 128

learning_rate = 0.1
epochs = 3

lstm_model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(lstm_model.parameters(), lr=learning_rate)

lstm_model.train()
losses = []
for epoch in range(epochs):
    epoch_loss = []
    for i in range(train_size):
        sent, label = getVal(i)
        
        inputs = Variable(torch.from_numpy(sent_to_tensor(sent)))
        target = Variable(torch.from_numpy(np.array([label])).long())
        
        lstm_model.zero_grad()
        lstm_model.hidden = lstm_model.init_hidden()
        
        outputs = lstm_model(inputs)
        loss = criterion(outputs, target)
        
        epoch_loss.append(loss.data[0])
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            print(i , 'samples processed')
            print('Avg loss after %d samples ' % i, np.average(epoch_loss))
    losses.append(np.average(epoch_loss))
    print('Avg Epoch Loss:', losses[epoch])
        

0 samples processed
Avg loss after 0 samples  0.649126768112
1000 samples processed
Avg loss after 1000 samples  0.775575342577
2000 samples processed
Avg loss after 2000 samples  0.740419247064
3000 samples processed
Avg loss after 3000 samples  0.727292080674
4000 samples processed
Avg loss after 4000 samples  0.721381831708
5000 samples processed
Avg loss after 5000 samples  0.716623214522
6000 samples processed
Avg loss after 6000 samples  0.71216655955
7000 samples processed
Avg loss after 7000 samples  0.708805617718
8000 samples processed
Avg loss after 8000 samples  0.704903775143
9000 samples processed
Avg loss after 9000 samples  0.700057807592


KeyboardInterrupt: 

In [105]:
def test_line(model , line):
    sent_tensor = Variable(torch.from_numpy(sent_to_tensor(line)))
    model.eval()
    model.hidden = model.init_hidden()
    output = model(sent_tensor)
    return output
    

In [108]:
def evaluate(lstm_model, x_test, y_test):
    correct = 0.0
    for i in range(len(x_test)):
        input = x_test[i]
        target = y_test.iloc[i]
#         print(input)
#         print(target)
#         break
        output = test_line(lstm_model, input[0])
        if output.topk(1)[1].data.numpy()[0][0] == target:
            correct += 1
#         if i % 100 == 0:
#             print(correct*1.0/ (i+1))
    print(correct*1.0/ len(x_test))
    

### Not even a complete epoch and we hit somewhere around 60%  

In [109]:
evaluate(lstm_model, x_test=X_test, y_test=y_test)

0.595
